In [1]:
import pandas as pd
import numpy as np
from random import random
import copy
SEED=13

In [2]:
b_real_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/b_sample.csv"
l_real_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/l_sample.csv"
b_syn_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/b_sample_syn.csv"
l_syn_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/l_sample_syn.csv"

In [3]:
#Import data
b_real=pd.read_csv(b_real_path, dtype=str) #base
l_real=pd.read_csv(l_real_path, dtype=str) #longitudinal 

In [4]:
def gen_toy_syn(df_org, r):
    #Generate synthetic data by shuffling the columns of certain portion of the input datasets
    # r is ratio of the records to be corrupted e..g 0.2 means 20% of the table is modified.
    df=copy.deepcopy(df_org)
    shuffled=df.sample(frac=1, ignore_index=True, random_state=SEED)
    r_rows=int(r*len(shuffled))
    corrupt=shuffled[0:r_rows].reset_index(drop=True)
    intact=shuffled[r_rows:].reset_index(drop=True)
    org_arr=corrupt.values
    n_col=org_arr.shape[1]
    new_arr=np.empty_like(org_arr)
    new_arr[:,0]=org_arr[:,0]
    for i in range(1,n_col):
        new_arr[:,i]=np.random.RandomState(seed=SEED).permutation(org_arr[:,i])
    corrupt=pd.DataFrame(new_arr, columns=corrupt.columns)
    syn=pd.concat([intact, corrupt],axis=0)
    return syn


In [5]:
b_syn=gen_toy_syn(b_real, 0.2)
b_syn.to_csv(b_syn_path, index=False)
l_syn=gen_toy_syn(l_real, 0.2)
l_syn.to_csv(l_syn_path, index=False)